In [ ]:
import requests
import pandas as pd
import os
os.makedirs("rechercheZenodo", exist_ok=True)

ACCESS_TOKEN = 'indiquer son TOKEN'
url = 'https://zenodo.org/api/records'

# Paramètres de la requête
params = {
    'q': 'Inria, Institut national de recherche en informatique et en automatique',
    'access_token': ACCESS_TOKEN,
    'sort': 'bestmatch',
    'size': 100,  # Limite à 100 résultats par page
    'publication_date': '*'
}

# Fonction pour récupérer les publications
def get_publications():
    publications = []
    page = 1
    while True:
        params['page'] = page
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            hits = data['hits']['hits']
            if not hits:
                break
            # Extraire les informations demandées
            for record in hits:
                creators = record['metadata'].get('creators', [])
                # Concaténation des informations des créateurs (nom, affiliation, orcid)
                creators_info = ', '.join([f"{creator.get('name', 'N/A')} ({creator.get('affiliation', 'N/A')}, {creator.get('orcid', 'N/A')})" for creator in creators])

                # Institution: Concaténer toutes les affiliations des auteurs, en évitant les None
                institutions = ', '.join([creator.get('affiliation') for creator in creators if creator.get('affiliation')])  # Inclut seulement les affiliations non None ou non vides

                # Récupérer le type de ressource si disponible
                publication_type = record['metadata'].get('resource_type', {}).get('type', 'N/A')

                # Récupération des sujets (si disponibles)
                subjects = record['metadata'].get('subjects', [])
                if subjects:
                    subjects_info = ', '.join([f"{subject.get('term', 'N/A')} (ID: {subject.get('identifier', 'N/A')}, Scheme: {subject.get('scheme', 'N/A')})" for subject in subjects])
                else:
                    subjects_info = 'N/A'

                # Récupérer les keywords (mots-clés) si disponibles
                keywords = record['metadata'].get('keywords', [])
                if keywords:
                    keywords_info = ', '.join(keywords)
                else:
                    keywords_info = 'N/A'

                # Créer l'URL de Zenodo en utilisant l'identifiant de l'enregistrement
                record_id = record['id']
                url_zenodo = f"https://zenodo.org/records/{record_id}"

                publications.append({
                    'title': record['metadata'].get('title', 'N/A'),
                    'publication_date': record['metadata'].get('publication_date', 'N/A'),
                    'doi': record['metadata'].get('doi', 'N/A'),
                    'creators': creators_info,
                    'publication_type': publication_type,
                    'access_right': record['metadata'].get('access_right', 'N/A'),
                    'institution': institutions if institutions else 'N/A',  # Toutes les affiliations
                    'subjects': subjects_info,  # Ajout des sujets formatés correctement
                    'keywords': keywords_info,  # Ajout des mots-clés (keywords)
                    'url_zenodo': url_zenodo  # URL Zenodo pour chaque publication
                })
            page += 1
        else:
            print(f"Erreur lors de la récupération des données: {response.status_code}")
            break
    return publications

# Récupérer les publications et les mettre dans un DataFrame
publications = get_publications()
df_publications = pd.DataFrame(publications)

df_publications


In [1]:
df_publications.to_csv("rechercheZenodo/Zenodo_recherche_publications.csv", index=False)

NameError: name 'df_publications' is not defined